# Telescope Beamforming for 3D Observational Data
This notebook simulates the behavior of a telescope array using beamforming. The signals from multiple telescopes are aligned to enhance the observation of a celestial object arriving from a particular direction. This technique is commonly used in radio astronomy and interferometry.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import chirp

# Simulation parameters
fs = 1000  # Sampling frequency (Hz)
duration = 1  # Duration of the observation (seconds)
t = np.linspace(0, duration, int(fs * duration), endpoint=False)  # Time vector

# Signal generation (simulate cosmic signal)
frequency = 100  # Frequency of the cosmic signal (Hz)
signal = chirp(t, f0=frequency, f1=frequency, t1=duration, method='linear')  # Simulated cosmic signal

# Telescope array parameters (3D positions)
num_telescopes = 4  # Number of telescopes in the array
positions = np.array([
    [0, 0, 0],  # Telescope 1 at origin
    [0.1, 0, 0],  # Telescope 2 at (0.1, 0, 0)
    [0, 0.1, 0],  # Telescope 3 at (0, 0.1, 0)
    [0, 0, 0.1]  # Telescope 4 at (0, 0, 0.1)
])

# Signal source direction (DOA) in spherical coordinates (theta, phi)
theta = np.pi / 4  # 45 degrees from the z-axis
phi = np.pi / 6  # 30 degrees from the x-axis

# Convert to Cartesian coordinates for direction vector
direction = np.array([
    np.sin(theta) * np.cos(phi),
    np.sin(theta) * np.sin(phi),
    np.cos(theta)
])

# Calculate time delays based on the source direction and telescope positions
sound_speed = 3e8  # Speed of electromagnetic waves (m/s)
delays = np.dot(positions, direction) / sound_speed  # Time delays in seconds
delays = np.round(delays * fs).astype(int)  # Convert to samples

# Simulate telescope signals with delays
telescope_signals = np.zeros((num_telescopes, len(t)))
for i in range(num_telescopes):
    delayed_signal = np.roll(signal, delays[i])
    telescope_signals[i, :] = delayed_signal

# Beamforming function for 3D observational data
def delay_and_sum(signals, delays):
    num_telescopes, num_samples = signals.shape
    beamformed_signal = np.zeros(num_samples)
    for i in range(num_telescopes):
        beamformed_signal += np.roll(signals[i, :], -delays[i])
    return beamformed_signal / num_telescopes

# Apply beamforming
beamformed_signal = delay_and_sum(telescope_signals, delays)

# Visualization of results
fig = plt.figure(figsize=(12, 8))

# 3D scatter plot of telescope positions
ax1 = fig.add_subplot(311, projection='3d')
ax1.set_title('Telescope Array Configuration')
ax1.scatter(positions[:, 0], positions[:, 1], positions[:, 2], c='r', label='Telescopes')
ax1.quiver(0, 0, 0, *direction, length=0.2, color='b', label='Source Direction')
ax1.set_xlabel('X [m]')
ax1.set_ylabel('Y [m]')
ax1.set_zlabel('Z [m]')
ax1.legend()

# Plot individual telescope signals
ax2 = fig.add_subplot(312)
ax2.set_title('Telescope Signals')
for i in range(num_telescopes):
    ax2.plot(t, telescope_signals[i, :], label=f'Telescope {i+1}')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Amplitude')
ax2.legend()

# Plot beamformed signal
ax3 = fig.add_subplot(313)
ax3.set_title('Beamformed Signal')
ax3.plot(t, beamformed_signal)
ax3.set_xlabel('Time [s]')
ax3.set_ylabel('Amplitude')

plt.tight_layout()
plt.show()
